In [1]:
from cuml.preprocessing.model_selection import train_test_split

import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
from cuml.datasets.classification import make_classification

from joblib import parallel_backend

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

# This will use all GPUs on the local host by default
cluster = LocalCUDACluster(threads_per_worker=1, ip="", dashboard_address="8002")
c = Client(cluster)

n_workers = 2

/home/jzedlewski/anaconda3/envs/cuml_dev/lib/python3.7/site-packages/treelite/gallery/__init__.py:7: FutureWarning: treelite.gallery.sklearn has been moved to treelite.sklearn. treelite.gallery.sklearn will be removed in version 1.1.
  FutureWarning)
/home/jzedlewski/anaconda3/envs/cuml_dev/lib/python3.7/site-packages/treelite/gallery/sklearn/__init__.py:9: FutureWarning: treelite.gallery.sklearn has been moved to treelite.sklearn. treelite.gallery.sklearn will be removed in version 1.1.
  FutureWarning)


In [2]:
from contextlib import contextmanager
import time

@contextmanager
def timed(name):
    t0 = time.time()
    yield
    t1 = time.time()
    print("..%-24s:  %8.4f" % (name, t1 - t0))

In [3]:
def inner_func():
    return X.sum()

def inner_func_explicit(x_in):
    return x_in.sum()

In [4]:
X, y = make_classification(n_samples=10_000_000, n_features=10, n_classes=2)

In [5]:
import joblib
with timed("dask-backend"):
    with parallel_backend("dask", n_jobs=n_workers, client=c):
        res_dask = joblib.Parallel(verbose=100)(
            joblib.delayed(inner_func)()
            for i in range(10)
        )

NameError: name 'n_workers' is not defined

In [ ]:
import joblib
with timed("dask-backend-ex-noscatter"):
    with parallel_backend("dask", n_jobs=n_workers, client=c):
        res_dask = joblib.Parallel(verbose=100)(
            joblib.delayed(inner_func_explicit)(X)
            for i in range(10)
        )

In [ ]:
import joblib
with timed("dask-backend-ex-scatter"):
    with parallel_backend("dask", n_jobs=n_workers, client=c, scatter=[X]):
        res_dask = joblib.Parallel(verbose=100)(
            joblib.delayed(inner_func_explicit)(X)
            for i in range(10)
        )

In [ ]:
with timed("loky-backend"):
    with parallel_backend("loky", n_jobs=n_workers):
        res_lok = joblib.Parallel(verbose=100)(
            joblib.delayed(inner_func)()
            for i in range(10)
        )

In [ ]:
with timed("loky-backend-ex"):
    with parallel_backend("loky", n_jobs=n_workers):
        res_loky = joblib.Parallel(verbose=100)(
            joblib.delayed(inner_func_explicit)(X)
            for i in range(10)
        )

In [ ]:
res = []
with timed("seq"):
    for i in range(10):
        res.append(inner_func_explicit(X))